# NT + ESM-2 ZERO-SHOT Ver


### 1) Split parquet into train/val/test (chr20/21 as test)


In [1]:
from split_data import main as split_main
from config import RAW_PARQUET
split_main(RAW_PARQUET)

Saved:
  train: 114568 -> d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\train.parquet
  val  : 20219 -> d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\val.parquet
  test : 4562 -> d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\test.parquet


### 2) Precompute embeddings (NT + ESM-2)


In [2]:
%env TOKENIZERS_PARALLELISM=false
from precompute_embeddings import main as emb_main
emb_main()

env: TOKENIZERS_PARALLELISM=false


c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
Loading NT...


Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading ESM-2...


...

[saved] d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\train_embeddings.pt
Embedding split d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\val.parquet (20219 rows)


100%|██████████| 5055/5055 [02:56<00:00, 28.71it/s]


[saved] d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\val_embeddings.pt
Embedding split d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\test.parquet (4562 rows)


100%|██████████| 1141/1141 [00:39<00:00, 28.68it/s]


[saved] d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\test_embeddings.pt


### 3) Train fusion classifier

Để theo dõi tiến độ train trong tensorboard:

- Mở terminal -> nhập: tensorboard --logdir runs -> truy cập vào link được in ra


In [1]:
from train import train
from config import LR, EPOCHS, BATCH_SIZE, PATIENCE, DROPOUT, SEED
import argparse

parser = argparse.ArgumentParser(description="Train fusion classifier with enhanced features.")
parser.add_argument("--lr", type=float, default=LR, help="Learning rate")
parser.add_argument("--epochs", type=int, default=EPOCHS)
parser.add_argument("--batch_size", type=int, default=BATCH_SIZE)
parser.add_argument("--patience", type=int, default=PATIENCE)
parser.add_argument("--dropout", type=float, default=DROPOUT)
parser.add_argument("--seed", type=int, default=SEED)
parser.add_argument("--log_dir", type=str, default="runs/experiment_1", help="TensorBoard log directory")

args = parser.parse_args([])

result = train(args)
result

c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


[1] Train Loss: 0.1736 | Val Loss: 0.1517 | Train Acc: 0.9328 | Val Acc: 0.9406
--> Saved best model checkpoint to d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\best_fusion_model.pt


[2] Train Loss: 0.1374 | Val Loss: 0.1599 | Train Acc: 0.9472 | Val Acc: 0.9415


[3] Train Loss: 0.1150 | Val Loss: 0.1640 | Train Acc: 0.9565 | Val Acc: 0.9435


[4] Train Loss: 0.0892 | Val Loss: 0.1870 | Train Acc: 0.9660 | Val Acc: 0.9440


[5] Train Loss: 0.0688 | Val Loss: 0.1984 | Train Acc: 0.9743 | Val Acc: 0.9434


[6] Train Loss: 0.0369 | Val Loss: 0.2804 | Train Acc: 0.9864 | Val Acc: 0.9426
Early stopping triggered.

--- Testing with Best Model ---


[TEST] Loss: 0.1555 | AUC: 0.9850 | MCC: 0.8770 | Acc: 0.9397


{'acc': 0.9397194385528564,
 'auc': 0.984959065914154,
 'mcc': 0.8769822716712952,
 'loss': 0.15545253942899018}

# NT + ESM-2 FINE-TUNED Ver
